In [126]:
#!pip install deap 

In [127]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


## Setup

In [130]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


class Setup:
    def __init__(self, params, dadosEntrada=None):

        #! Parametros JSON
        self.params = params
        self.DADOS_ENTRADA = dadosEntrada  # Dados da entrada do usuario

        self.CXPB = params["CROSSOVER"]
        self.MUTPB = params["MUTACAO"]
        self.NGEN = params["NUM_GENERATIONS"]
        self.POP_SIZE = params["POP_SIZE"]
        self.SIZE_INDIVIDUAL = params["IND_SIZE"]
        self.TAXA_GENERATION = params["RCE_REPOPULATION_GENERATIONS"]
        self.CROSSOVER, self.MUTACAO, self.NUM_GENERATIONS, self.POPULATION_SIZE = (
            self.CXPB,
            self.MUTPB,
            self.NGEN,
            self.POP_SIZE,
        )

        self.delta = params["DELTA"]
        self.porcentagem = params["PORCENTAGEM"]

        #! Parâmetros do algoritmo de Rastrigin
        self.evaluations = 0
        self.num_repopulation = int(self.NUM_GENERATIONS * self.TAXA_GENERATION)
        self.type = params["type"].lower()
        if self.type == "maximize":
            print("Método escolhido: Maximizar")
            creator.create("Fitness", base.Fitness, weights=(1.0,))
        else:
            print("Método escolhido: Minimizar")
            creator.create("Fitness", base.Fitness, weights=(-1.0,))

        self.dataset = {}
        self.beta_constants = {}

        #!Criando individuo pelo deap com seus atributos
        self.toolbox = base.Toolbox()

        # Função de fitness de minimização
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

        # parâmetros
        creator.create(
            "Individual", list, fitness=creator.FitnessMin, rce=str, index=int
        )

        # Variavel de decisão
        self.toolbox.register("attribute", random.uniform, -5.12, 5.12)

        # registrando os individuos
        self.toolbox.register(
            "individual",
            tools.initRepeat,
            creator.Individual,
            self.toolbox.attribute,
            n=self.SIZE_INDIVIDUAL,
        )

        #! individuos com dados de entrada
        if self.DADOS_ENTRADA:
            creator.create(
                "FrameworkIndividual",
                list,
                fitness=creator.FitnessMin,
                tipo=self.DADOS_ENTRADA[0],
                rce=str,
            )
            self.toolbox.register(
                "varDecision",
                random.uniform,
                self.DADOS_ENTRADA[2][0],
                self.DADOS_ENTRADA[2][1],
            )
            self.toolbox.register(
                "newIndividual",
                tools.initRepeat,
                creator.FrameworkIndividual,
                self.toolbox.varDecision,
                n=self.DADOS_ENTRADA[1],
            )

            self.toolbox.register(
                "FrameworkPopulation",
                tools.initRepeat,
                list,
                self.toolbox.newIndividual,
            )

        #! paramentos evolutivos
        self.toolbox.register(
            "population", tools.initRepeat, list, self.toolbox.individual
        )

        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        self.toolbox.register("evaluate", self.evaluate_fitness)

    def evaluate_fitness(self, individual):
        if self.type == "minimaze":
            result = minimize(
                self.rastrigin, x0=np.zeros(self.SIZE_INDIVIDUAL), method="BFGS"
            )
            fitness_value = result.fun
        return fitness_value

    def gerarDataset(self, excel):
        df = pd.read_excel(excel)
        print(df.columns)
        self.dataset = {
            "CXPB": self.CROSSOVER,
            "TAXA_MUTACAO": self.MUTACAO,
            "NUM_GEN": self.NUM_GENERATIONS,
            "POP_SIZE": self.POPULATION_SIZE,
            "IND_SIZE": self.SIZE_INDIVIDUAL,
            "evaluations": self.evaluations,
            "NUM_REPOPULATION": self.num_repopulation,
        }

    def rastrigin(self, individual):
        self.evaluations += 1
        rastrigin = 10 * self.SIZE_INDIVIDUAL
        for i in range(self.SIZE_INDIVIDUAL):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

    def rastrigin_decisionVariables(self, individual, decision_variables):
        self.evaluations += 1
        rastrigin = 10 * len(decision_variables)
        for i in range(len(decision_variables)):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

    def rosenbrock(self, x):
        return np.sum(100 * (x[1:] - x[:-1] ** 2) ** 2 + (1 - x[:-1]) ** 2)

    def globalSolutions(self):
        n_dimensions = 2

        try:
            rastrigin_result = minimize(
                self.rastrigin, x0=np.zeros(n_dimensions), method="BFGS"
            )
            rastrigin_minimum = rastrigin_result.fun
            rastrigin_solution = rastrigin_result.x

            rosenbrock_result = minimize(
                self.rosenbrock, x0=np.zeros(n_dimensions), method="BFGS"
            )
            rosenbrock_minimum = rosenbrock_result.fun
            rosenbrock_solution = rosenbrock_result.x

        except Exception as e:
            print("Erro ao tentar encontrar o ótimo global das funções: ", e)

        print("\n\nÓtimo global da função Rastrigin: ", rastrigin_minimum)
        print("Solução: ", rastrigin_solution)
        print()
        print("Ótimo global da função Rosenbrock: ", rosenbrock_minimum)
        print("Solução: ", rosenbrock_solution)

## Algoritimo Evolutivo

In [131]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


class AlgoritimoEvolutivoRCE:
    def __init__(self, setup):
        self.setup = setup
        self.stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)

        self.logbook = tools.Logbook()
        self.hof = tools.HallOfFame(1)
        self.POPULATION = self.setup.toolbox.population(n=self.setup.POP_SIZE)
        self.hof.update(self.POPULATION)

        if self.setup.DADOS_ENTRADA:
            self.POP_OPTIMIZATION = self.setup.toolbox.population(n=self.setup.POP_SIZE)

        self.pop_RCE = []
        self.best_solutions_array = []
        self.best_individual_array = []
        self.allIndividualValuesArray = []
        self.data = {}
        self.repopulation_counter = 0
        self.allFitnessValues = {}
        self.validateCounter = 0
        self.CONJUNTO_ELITE_RCE = set()

    def registrarDados(self, generation):

        # Registrar estatísticas e melhores soluções
        for ind in self.POPULATION:
            avg_fitness_per_generation = np.mean(ind.fitness.values)
            std_deviation = np.std(ind.fitness.values)

        #! PEgandos os dados e colocando no df
        self.data = {
            "Generations": generation + 2,
            "Variaveis de Decisão": self.hof[0],
            "Evaluations": self.setup.evaluations,
            "Ind Valido": self.hof[0].fitness.valid,
            "Best Fitness": self.hof[0].fitness.values,
            "Media": avg_fitness_per_generation,
            "Desvio Padrao": std_deviation,
        }

        self.best_individual_array.append(self.data)

        self.visualizarPopAtual(generation, [avg_fitness_per_generation, std_deviation])

    def checkClonesInPop(self, ind, new_pop):
        is_clone = False
        for other_ind in new_pop:
            if (
                ind == other_ind
                and sum(ind) == sum(other_ind)
                and ind.index != other_ind.index
            ):
                is_clone = True
                break
        return is_clone

    def generateInfoIndividual(self, new_pop, generation):
        ind_array = []

        for i, ind in enumerate(new_pop):
            # print(f"Index[{ind.index}] - ind_variables {ind} \n Fitness = {ind.fitness.values} ")

            ind.index = i

            ind_info = {
                "Generations": generation,
                "index": ind.index,
                "Variaveis de Decisão": ind,
                "Fitness": ind.fitness.values[0],
                "RCE": ind.rce,
                "Diversidade": np.sum(ind),
            }

            # Adicionar a informação de clone ao dicionário
            # is_clone = self.checkClonesInPop(ind, new_pop)
            # ind_info["CLONE"] = "SIM" if is_clone else "NAO"

            ind_array.append(ind_info)

        return ind_array

    def show_ind_df(self, array, text):
        df = pd.DataFrame(array)
        print(text)
        display(df.head(40))

        # contar quantos SIM na coluna CLONE se a coluna RCE for SIM
        # display(df[df["RCE"] != ""]["CLONE"].value_counts())

    def criterio1(self, new_pop, porcentagem, k=30):
        """Seleciona os candidatos ao conjunto elite com base nas diferenças percentuais de aptidão."""
        self.cout(f"CRITÉRIO 1 RCE - Selecionando candidatos ao conjunto elite")
        elite_individuals = []

        # Ordenar a população em ordem crescente de aptidão e selecionar os k primeiros indivíduos
        sorted_population = sorted(self.POPULATION, key=lambda x: x.fitness.values[0])

        # Obter o melhor indivíduo (HOF) da população
        best_ind = new_pop[0]
        best_fitness = best_ind.fitness.values[0]
        max_difference = (1 + porcentagem) * best_fitness

        # Selecionar com as menores diferenças percentuais
        for ind in sorted_population:
            if ind.fitness.values[0] <= max_difference:
                elite_individuals.append(ind)
            else:
                break  # Parar a seleção quando a diferença percentual for maior que o limite

        # Colocando na pop aleatória
        print(
            f"Calculando percentual de {porcentagem*100}% com base no melhor fitness = {best_fitness} e pegando os {len(elite_individuals)} melhores.\n Porcentagem de {best_fitness} = {max_difference} "
        )
        for i, ind in enumerate(elite_individuals):
            new_pop[i] = self.setup.toolbox.clone(ind)
            new_pop[i].rce = "SIM_1"

        return elite_individuals

    def criterio2_alternative(self, ind_selecionados, delta=6):
        """Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta'."""
        self.cout(
            f"CRITÉRIO 2 - Comparar as variáveis de decisão de cada indivíduo e verificar valores superiores a 'delta' = {delta}."
        )
        self.CONJUNTO_ELITE_RCE.clear()

        for i in range(len(ind_selecionados)):

            #    todo   Calcular Diff - diferença entre as variáveis de indivíduo e individuo lista

            diff = np.array(ind_selecionados[i]) - np.array(ind_selecionados[0])

            # todo retornar true ou false caso ind seja diferente para colocar no array correto
            if sum(diff) > delta:  # ind diferente
                if (ind_selecionados[i] not in self.pop_RCE) and (
                    ind_selecionados[i] not in ind_selecionados[0]
                ):
                    self.pop_RCE.append(self.POP_OPTIMIZATION[i])
                    self.CONJUNTO_ELITE_RCE.add(tuple(self.POP_OPTIMIZATION[i]))
                    ##print("Delta= ", sum(diff))

        if not self.pop_RCE:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        print("Tamanho Elite = ", len(self.CONJUNTO_ELITE_RCE))

        return self.pop_RCE

    def newCriterio(self, population):
        self.CONJUNTO_ELITE_RCE.clear()
        self.pop_RCE = []

        def criterio1_reduzido(population):
            #! critério 1 e obtém os N melhores com 30% do valor do melhor fitness

            best_ind = self.elitismoSimples(population)[0]
            best_fitness = best_ind.fitness.values[0]
            max_difference = (1 + self.setup.porcentagem) * best_fitness
            print(
                f"Fitness ({self.setup.porcentagem * 100})% = {round(max_difference,3)}"
            )
            return max_difference, best_ind

        self.cout("Criterio 1 - Pegando o valor máximo de Fitness para selecionar individuos")
        max_difference, best_ind = criterio1_reduzido(population)
        self.pop_RCE.append(best_ind)

        def calculaDiff(ind, lista):
            count = 0

            # calcula a diferença entre o ind selecionado e o pessoal do RCE
            for i in range(0,len(lista)):
                diff = abs(np.array(ind) - np.array(lista[i]))
                array = list(diff)

                # pegando quantos valores nao sao nulos  -> caso tenha valor limite
                if (sum(array)> 0.0):
                    for value in array:
                        if value != 0.0:
                            count += 1

                    # se o contador for maior que delta
                    if count >= self.setup.delta:
                        print("\nArray diferente")

                        print(" diff", array)

                        print("Var decision diferentes = ", count)
                        return True

                    else:
                        return False # sem diversidade suficiente
                else:
                    return False # clone: Variaveis iguais

        self.cout(f"New - CRITÉRIO 2 RCE ")
        for ind in population:
            # criterio 1
            if ind.fitness.values[0] <= max_difference:
                # criterio 2
                diferente = calculaDiff(ind, self.pop_RCE) # delta como valor limite
                if diferente:
                    if ind not in self.pop_RCE:
                        self.pop_RCE.append(ind)
                        self.CONJUNTO_ELITE_RCE.add(tuple(ind))

        if len(self.pop_RCE) == 1:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("\nTamanho Elite = ", len(self.pop_RCE))
        print("Tamanho Elite = ", len(self.CONJUNTO_ELITE_RCE))

        return self.pop_RCE

    def aplicar_RCE(self, generation, current_population):

        #! a - Cria uma pop aleatória (eliminando a pop aleatória criada na execução anterior do RCE)
        new_pop = self.setup.toolbox.population(
            n=self.setup.POP_SIZE
        )  # retorna uma pop com lista de individuos de var de decisão

        # Avaliar o fitness da população atual
        self.avaliarFitnessIndividuos(current_population)
        self.calculateFitnessGeneration(current_population)

        #! b - Coloca o elite hof da pop anterior  no topo (0)
        pop = self.elitismoSimples(current_population)
        print(
            f"Elitismo HOF Index[{pop[0].index}] {pop[0]} \n Fitness = {pop[0].fitness.values} | Diversidade = {sum(pop[0])}"
        )  # pop[0] é o melhor individuo HOF
        new_pop[0] = self.setup.toolbox.clone(pop[0])

        #! Critério 2 usando este array e vai colocando os indivíduos selecionados pelo critério 2 na pop aleatória (passo a)
        ind_diferentes_var = self.newCriterio(
            current_population, 
        )

        # cOLOCANDO ATRIBUTOS
        for i, ind in enumerate(ind_diferentes_var, start=0):
            new_pop[0].rce = "HOF"
            if i > 0:
                new_pop[i] = self.setup.toolbox.clone(ind)
                new_pop[i].rce = "SIM"

        #! Criterio 3 retorna pop aleatória modificada (com hof + rce + Aleatorio)
        self.calculateFitnessGeneration(new_pop)
        self.cout(f"CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] ")
        conjunto_elite = self.generateInfoIndividual(new_pop, generation)
        self.show_ind_df(conjunto_elite, "Individuos da nova população aleatória")
        return new_pop

    def elitismoSimples(self, pop):
        self.hof.update(pop)
        pop[0] = self.setup.toolbox.clone(self.hof[0])
        return pop

    def criterio2(self, elite_individuals, delta):
        """Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta'."""
        self.cout(
            f"CRITÉRIO 2 - Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta' = {delta}."
        )
        self.pop_RCE = []
        self.CONJUNTO_ELITE_RCE.clear()

        for i in range(len(elite_individuals)):
            current_individual = elite_individuals[i]
            is_diferente = False

            for j in range(i + 1, len(elite_individuals)):
                other_individual = elite_individuals[j]
                diff_counter = 0

                for var_index in range(len(current_individual)):
                    current_var = current_individual[var_index]
                    other_var = other_individual[var_index]

                    if abs(current_var - other_var) > delta:
                        # print(abs(current_var - other_var))
                        diff_counter += 1

                if diff_counter >= 1:
                    is_diferente = True

            if is_diferente:
                # print(f"Indivíduo do tipo {type(current_individual)} VAR({current_individual})\n diferente! adicionado à nova população.")
                self.pop_RCE.append(current_individual)
                self.CONJUNTO_ELITE_RCE.add(tuple(current_individual))

        if not self.pop_RCE:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        return self.pop_RCE

    def avaliarFitnessIndividuos(self, pop):
        """Avaliar o fitness dos indivíduos da população atual."""
        fitnesses = map(self.setup.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            if ind.fitness.values:
                ind.fitness.values = [fit]

    def calculateFitnessGeneration(self, new_pop):
        # Calculando o fitness para geração
        for ind in new_pop:
            if not ind.fitness.valid:
                fitness_value = self.setup.toolbox.evaluate(ind)
                ind.fitness.values = (fitness_value,)

    def checkDecisionVariablesAndFitnessFunction(
        self, decision_variables, fitness_function
    ):

        # Verificar se as variáveis de decisão e a função de fitness foram fornecidas
        if decision_variables is None and fitness_function is None:
            # Gerar variáveis de decisão aleatórias para os indivíduos
            decision_variables = [
                random.random() for _ in range(self.setup.SIZE_INDIVIDUAL)
            ]

            # Definir a função de fitness padrão como a função Rastrigin
            fitness_function = self.setup.rastrigin_decisionVariables

        if decision_variables is None or fitness_function is None:
            if not hasattr(self, "decision_variables") or not hasattr(
                self, "fitness_function"
            ):
                raise ValueError(
                    "Variáveis de decisão e função de fitness não definidas. Use set_decision_variables_and_fitness_function primeiro."
                )
        else:
            self.decision_variables = decision_variables
            self.fitness_function = fitness_function

            # Definir a função de fitness com base na função fornecida
            def fitness_func(individual):
                return self.fitness_function(individual, self.decision_variables)

            # Registrar a função de fitness no toolbox
            self.setup.toolbox.register("evaluate", fitness_func)

    #! Main LOOP
    def run(self, RCE=False, decision_variables=None, fitness_function=None, num_pop=0):

        if self.setup.DADOS_ENTRADA:
            population = [self.POPULATION, self.POP_OPTIMIZATION]
        else:
            population = [self.POPULATION]

        # Avaliar o fitness da população inicial
        self.avaliarFitnessIndividuos(population[num_pop])

        # Selecionando as variaveis de decisao e afuncao objeti
        self.checkDecisionVariablesAndFitnessFunction(
            decision_variables, fitness_function
        )

        #! Loop principal através das gerações
        for current_generation in range(self.setup.NGEN):

            # Selecionar os indivíduos para reprodução
            offspring = self.setup.toolbox.select(
                population[num_pop], k=len(population[num_pop])
            )

            # Clone the selected individuals
            offspring = [self.setup.toolbox.clone(ind) for ind in offspring]

            # Aplicar crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.setup.CXPB:
                    self.setup.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values

            # Aplicar mutação
            for mutant in offspring:
                if random.random() < self.setup.MUTPB:
                    self.setup.toolbox.mutate(mutant)
                    del mutant.fitness.values

            #  Avaliar o fitness dos novos indivíduos
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(self.setup.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = [fit]

            #! Aplicar RCE
            if RCE and ((current_generation + 1) % self.setup.num_repopulation == 0):
                print("")
                self.cout(
                    f"RCE being applied! - Generation = {current_generation + 1} ",
                )
                #! f - copia pop aleatória modificada retornada para pop atual
                new_population = self.aplicar_RCE(
                    current_generation + 1, population[num_pop]
                )
                # print("\nPopulação gerada pelo RCE\n", population[num_pop])
                population[num_pop][:] = new_population
            else:
                population[num_pop][:] = offspring

            # Registrar estatísticas no logbook
            self.elitismoSimples(population[num_pop])
            self.registrarDados(current_generation)
            record = self.stats.compile(population[num_pop])
            self.logbook.record(gen=current_generation, **record)

        # Retornar população final, logbook e elite
        return population[num_pop], self.logbook, self.hof[0]

    def visualizarPopAtual(self, geracaoAtual, stats):

        for i in range(len(self.POPULATION)):
            datasetIndividuals = {
                "Generations": geracaoAtual + 1,
                "index": i,
                "Variaveis de Decisão": self.POPULATION[i],
                "Fitness": self.POPULATION[i].fitness.values,
                "Media": stats[0],
                "Desvio Padrao": stats[1],
                "RCE": " - ",
            }
            self.allIndividualValuesArray.append(datasetIndividuals)

    def cout(self, msg):
        print(
            "\n=========================================================================================================="
        )
        print("\t", msg)
        print(
            "==========================================================================================================\n"
        )

## Exemplo

In [141]:
import random
from deap import base, creator, tools
from typing import List, Callable, Optional
import numpy as np


class CustomIndividual(list):
    def __init__(
        self,
        tipo: str,
        quantidade_var_decision: int,
        limite_var: List[Optional[float]],
        *args
    ):
        super().__init__(*args)
        self.tipo = tipo
        self.quantidade_var_decision = quantidade_var_decision
        self.limite_var = limite_var
        self.index = None
        self.rce = None


# Função de avaliação para o problema de otimização Rastrigin
def rastrigin_fitness(individual: CustomIndividual) -> float:
    rastrigin = 10 * len(individual)
    for i in range(len(individual)):
        rastrigin += individual[i] ** 2 - 10 * (np.cos(2 * np.pi * individual[i]))
    return (rastrigin,)


# Exemplo de configuração do DEAP
def configure_deap(
    tipo: str,
    quantidade_var_decision: int,
    limite_var: List[Optional[float]],
):
    creator.create("Fitness", base.Fitness, weights=(-1.0,))
    creator.create("Individual", CustomIndividual, fitness=creator.Fitness, rce = str, index = int)

    toolbox = base.Toolbox()

    # Registro de função para inicializar os atributos dos indivíduos
    if tipo == "int":
        toolbox.register("attribute", random.randint, limite_var[0], limite_var[1])
    elif tipo == "float":
        toolbox.register("attribute", random.uniform, limite_var[0], limite_var[1])
    elif tipo == "binario":
        toolbox.register("attribute", random.randint, 0, 1)
    else:
        raise ValueError("Tipo de variável inválido")

    # Registro da função de avaliação (fitness)
    toolbox.register("evaluate", rastrigin_fitness)

    # Função para criar um indivíduo com atributos personalizados
    def create_individual(
        tipo: str, quantidade_var_decision: int, limite_var: List[Optional[float]]
    ):
        individual = creator.Individual(
            tipo,
            quantidade_var_decision,
            limite_var,
            [toolbox.attribute() for _ in range(quantidade_var_decision)],
        )
        individual.tipo = tipo
        individual.quantidade_var_decision = quantidade_var_decision
        individual.limite_var = limite_var
        return individual

    # Registro da função de criação de indivíduos
    toolbox.register(
        "individual", create_individual, tipo, quantidade_var_decision, limite_var
    )
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    return toolbox


# Exemplo de uso
tipo = "int"  # Pode ser "int", "float" ou "binario"
quantidade_var_decision = 5
limite_var = [1.0, 10]  

my_toolbox = configure_deap(
    tipo,
    quantidade_var_decision,
    limite_var,
)

# Criação de um indivíduo
ind = my_toolbox.individual()
print(type(ind))

print("Indivíduo:", ind)
print("Tipo:", ind.tipo)
print("Quantidade de Variáveis de Decisão:", ind.quantidade_var_decision)
print("Limites Variáveis:", ind.limite_var)
print("Índice:", ind.index)  # Pode ser None
print("RCE:", ind.rce)  # Pode ser None

# Avaliação do indivíduo usando a função Rastrigin
fitness = my_toolbox.evaluate(ind)
print("Fitness do Indivíduo:", fitness[0])


newPop = my_toolbox.population(n=10)
print("\nPopulaçao gerada\n", newPop)

<class 'deap.creator.Individual'>
Indivíduo: [6, 5, 2, 9, 6]
Tipo: int
Quantidade de Variáveis de Decisão: 5
Limites Variáveis: [1.0, 10]
Índice: 0
RCE: 
Fitness do Indivíduo: 182.0

Populaçao gerada
 [[2, 10, 5, 9, 2], [3, 3, 1, 8, 9], [3, 5, 9, 1, 3], [1, 7, 9, 5, 2], [2, 3, 5, 1, 8], [6, 10, 5, 9, 2], [8, 3, 8, 8, 7], [4, 8, 8, 10, 1], [1, 10, 6, 4, 7], [2, 1, 7, 2, 9]]


In [133]:
# Exemplo dados tubina eolica
#! Carregar os dados
turbina = pd.read_csv(
    r"/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/assets/datasets/T1.csv"
)
turbina.columns = [
    "Data/hora",
    "ActivePower(kW)",
    "WindSpeed(m/s)",
    "Curva_Teórica(KWh)",
    "Direção do Vento",
]
display(turbina.head())

X = turbina[['WindSpeed(m/s)']].values
y = turbina[['ActivePower(kW)']].values
data_hora = turbina[['Data/hora']].values
curva = turbina[['Curva_Teórica(KWh)']].values
vento = turbina[['Direção do Vento']].values

print(X.shape,y.shape)


def rastrigin_decisionVariables( individual, decision_variables):
    rastrigin = 10 * len(decision_variables)
    for i in range(len(decision_variables)):
        rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
    return rastrigin

turbina = ClassIndividual.from_required(
    tipo="float",
    quantidade_var_decision=5,
    limite_var=[-5.12, 5.12],
)

turbina.name_prob = "Exemplo Turbina"
turbina.Fitness = rastrigin_decisionVariables
print(turbina)
print(type(turbina))

,Data/hora,ActivePower(kW),WindSpeed(m/s),Curva_Teórica(KWh),Direção do Vento
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


(50530, 1) (50530, 1)
name_prob='Exemplo Turbina' index=None tipo='float' quantidade_var_decision=5 limite_var=[-5.12, 5.12] Fitness=<function rastrigin_decisionVariables at 0x71e6c7c10cc0> RCE=None diversidade=None
<class '__main__.ClassIndividual'>


In [134]:
## problema da mochila - combinatria do tipo binario

# SA -> https://github.com/brunodeathayde/SA_TSP/blob/main/main_SA.py

#evolution
prob_dict = {
    "tipo": "binario",
    "num_var": 8,
    "lim_var": [1,12],
    "fitness": 0,
}

#problema
multa = 1000
pop = 10
capacidade_mochila = 15
quantidade_itens = [1,8,3,12,7,5,6,3]
lucro = [2,5,3,10,8,1,3,4]
fitness = lambda x: (capacidade_mochila - sum(quantidade_itens[x])) * multa * sum(lucro)

#print(fitness(2))

In [135]:
params = load_params(
    "/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/db/parameters.json"
)

new_setup = SetupInd(params)

pop = new_setup.create_population()


db_ind: List[ClassIndividual] = []


Método escolhido: Minimizar


/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


## Main

In [136]:
if __name__ == "__main__":
    tempo1 = time.time()

    # Setup
    params = load_params(
        r"/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/db/parameters.json"
    )
    setup = Setup(params)
    alg = AlgoritimoEvolutivoRCE(setup)
    # data_visual = DataExploration()

    pop_with_repopulation, logbook_with_repopulation, best_variables = alg.run(
        RCE=True,
        #fitness_function=rastrigin_decisionVariables,
        #decision_variables=(X, y,data_hora,curva,vento),
    )

    print("\n\nEvolução concluída  - 100%")

    # Salvar resultados
    alg.cout("---> RESULTADOS OBTIDOS")
    # data_visual.displayDataFrameInfo(alg.allIndividualValuesArray, "Todos Individuos Gerados")
    # data_visual.show_conjuntoElite(alg.pop_RCE)

    # Visualização dos resultados
    alg.cout("VISUALIZANDO OS RESULTADOS")
    # data_visual.show_rastrigin_benchmark(logbook_with_repopulation,best_variables)
    # data_visual.visualize(
    #   logbook_with_repopulation, pop_with_repopulation, repopulation=True
    # )
    # data_visual.statistics_per_generation_df(logbook_with_repopulation)

    tempo2 = time.time()
    print(f"Total execution time: {round(tempo2-tempo1,2)} seconds")

Método escolhido: Minimizar


	 RCE being applied! - Generation = 20 

Elitismo HOF Index[0] [-0.051702210725356856, -2.9829981835219828, -0.8817932382496902, -0.07961009450386669, -1.036277255257633, 0.08661658282648688, -0.13043141585776752, -1.963022789225807, 0.04359585238053487, -1.873426668257364] 
 Fitness = (31.1055393770506,) | Diversidade = -8.869049420392447

	 Criterio 1 - Pegando o valor máximo de Fitness para selecionar individuos

Fitness (30.0)% = 40.437

	 New - CRITÉRIO 2 RCE 

Nenhum indivíduo atende aos critérios. :( 

Tamanho Elite =  1
Tamanho Elite =  0

	 CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] 

Individuos da nova população aleatória


,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,20,0,"[-0.051702210725356856, -2.9829981835219828, -...",31.105539,HOF,-8.869049
1,20,1,"[2.8670812945156694, -2.711858391125627, -0.35...",156.176300,,-0.620261
2,20,2,"[-0.6554510079161489, 1.0921074498330006, -3.0...",169.669420,,-18.659090
3,20,3,"[3.1482138891815845, 0.5467556549736488, -3.43...",213.763814,,4.767928
4,20,4,"[-0.7649879880719785, 4.9999058096867115, 0.55...",184.374868,,-1.232682
5,20,5,"[-1.7194191416172373, 1.1809780116156592, -1.2...",215.102192,,-15.186393
6,20,6,"[4.106673674392069, 1.167921027801012, -0.8261...",145.501085,,-3.265984
7,20,7,"[1.5768745342262624, 2.331295846255075, -2.051...",199.998208,,25.795146
8,20,8,"[3.004606639226556, -3.7008179579384137, 4.472...",231.070956,,9.004922
9,20,9,"[-0.23494375297880854, 4.783888254390267, -1.8...",147.122127,,13.370554




	 RCE being applied! - Generation = 40 

Elitismo HOF Index[0] [-0.051702210725356856, 0.14256766362101647, 0.03787863096601418, 0.013106532712567898, -1.036277255257633, 0.08661658282648688, -0.08724207708074319, -0.08635548258364256, 0.04359585238053487, 0.08862091670142291] 
 Fitness = (12.208692115728132,) | Diversidade = -0.8491908464393323

	 Criterio 1 - Pegando o valor máximo de Fitness para selecionar individuos

Fitness (30.0)% = 15.871

	 New - CRITÉRIO 2 RCE 


Array diferente
 diff [0.0, 2.1696247675281666, 0.0, 0.0, 0.0, 0.0, 0.04318933877702433, 0.0, 0.0, 0.0]
Var decision diferentes =  2

Array diferente
 diff [0.0, 2.1696247675281666, 0.0, 0.0, 0.0, 0.0, 0.04318933877702433, 0.0, 0.0, 0.0]
Var decision diferentes =  2

Array diferente
 diff [0.0, 2.1005170817040746, 0.0, 0.0, 0.0, 0.0, 0.04318933877702433, 0.0, 0.0, 0.0]
Var decision diferentes =  2

Array diferente
 diff [0.0, 2.1696247675281666, 0.0, 0.0, 0.0, 0.0, 0.04318933877702433, 0.0, 0.0, 0.0]
Var decision d

,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,40,0,"[-0.051702210725356856, 0.14256766362101647, 0...",12.208692,HOF,-0.849191
1,40,1,"[-0.051702210725356856, -2.02705710390715, 0.0...",14.409169,SIM,-3.062005
2,40,2,"[-0.051702210725356856, -1.9579494180830583, 0...",14.336627,SIM,-2.992897
3,40,3,"[-3.9053909819009798, -0.27374032729371756, 3....",218.118555,,-8.761890
4,40,4,"[2.9230552337464877, -0.4246307786347776, 1.12...",171.043114,,-3.613921
5,40,5,"[-0.6755754821677371, 0.5900783972710926, -2.7...",202.134018,,-5.330172
6,40,6,"[2.2656457190682477, 0.6123607353016824, 2.285...",176.653540,,-3.324000
7,40,7,"[2.4145478892749432, 0.4174203864109822, 1.670...",185.351515,,-1.521491
8,40,8,"[-4.7572408284630505, 0.5544094602925433, -0.2...",152.152269,,-9.297825
9,40,9,"[-2.7594523297732962, 3.8036677027116044, -2.4...",184.239674,,5.364557




	 RCE being applied! - Generation = 60 

Elitismo HOF Index[0] [-0.051702210725356856, 0.04644423627278822, 0.03787863096601418, 0.013106532712567898, -1.036277255257633, -0.021428397009881195, -0.08724207708074319, -0.08635548258364256, 0.04359585238053487, -0.05203948913284173] 
 Fitness = (6.515195926087795,) | Diversidade = -1.1940196594581933

	 Criterio 1 - Pegando o valor máximo de Fitness para selecionar individuos

Fitness (30.0)% = 8.47

	 New - CRITÉRIO 2 RCE 


Array diferente
 diff [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.040426136710919, 0.0, 0.0, 0.14066040583426465]
Var decision diferentes =  2

Array diferente
 diff [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.040426136710919, 0.0, 0.0, 0.14066040583426465]
Var decision diferentes =  2

Array diferente
 diff [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.040426136710919, 0.0, 0.0, 0.14066040583426465]
Var decision diferentes =  2

Array diferente
 diff [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.040426136710919, 0.0, 0.0, 0.14066040583426465]
Var decision difer

...



	 RCE being applied! - Generation = 80 

Elitismo HOF Index[0] [-0.051702210725356856, 0.04644423627278822, 0.03787863096601418, 0.013106532712567898, -1.036277255257633, -0.00035403115979279823, 0.9531840596301759, -0.08635548258364256, 0.04359585238053487, -0.05203948913284173] 
 Fitness = (6.289587419533479,) | Diversidade = -0.13251915689718574

	 Criterio 1 - Pegando o valor máximo de Fitness para selecionar individuos

Fitness (30.0)% = 8.176

	 New - CRITÉRIO 2 RCE 

Nenhum indivíduo atende aos critérios. :( 

Tamanho Elite =  1
Tamanho Elite =  0

	 CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] 

Individuos da nova população aleatória


,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,80,0,"[-0.051702210725356856, 0.04644423627278822, 0...",6.289587,HOF,-0.132519
1,80,1,"[1.3744531011161003, -3.9838591485964288, -3.6...",162.431606,,-6.261978
2,80,2,"[0.4368489656874601, 1.6372099486683318, 2.153...",180.501750,,-0.693621
3,80,3,"[1.4123850241407858, -3.1058052290982414, -2.1...",159.493874,,-7.658065
4,80,4,"[-5.024132339169275, -2.1102676234424487, 1.10...",146.750023,,-1.489726
5,80,5,"[1.0187090861611159, 2.0413979503837156, 3.935...",205.051430,,10.498790
6,80,6,"[0.2706349713459364, 3.7828826444215897, 4.280...",171.072826,,11.023463
7,80,7,"[0.5757325181859274, 5.077933556336599, -1.388...",199.662302,,15.290513
8,80,8,"[4.2449359537156655, -4.678528988305516, 3.354...",186.820060,,0.506148
9,80,9,"[-3.2875057271180106, -1.000606672292192, 3.69...",181.649280,,7.048125




	 RCE being applied! - Generation = 100 

Elitismo HOF Index[0] [-0.051702210725356856, 0.04644423627278822, 0.03787863096601418, 0.013106532712567898, -1.035426518597169, -0.00035403115979279823, 0.9531840596301759, 0.04029165476927943, -0.023996398856778928, -0.05203948913284173] 
 Fitness = (4.8918490055061685,) | Diversidade = -0.07261353412111357

	 Criterio 1 - Pegando o valor máximo de Fitness para selecionar individuos

Fitness (30.0)% = 6.359

	 New - CRITÉRIO 2 RCE 


Array diferente
 diff [0.9429052107546444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0675922512373138, 0.0]
Var decision diferentes =  2

Tamanho Elite =  2
Tamanho Elite =  1

	 CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] 

Individuos da nova população aleatória


,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,100,0,"[-0.051702210725356856, 0.04644423627278822, 0...",4.891849,HOF,-0.072614
1,100,1,"[-0.9946074214800013, 0.04644423627278822, 0.0...",5.621830,SIM,-0.947926
2,100,2,"[-2.7282630960494205, -3.4110647309317472, -2....",248.797653,,-6.095588
3,100,3,"[-0.5866513809259217, -2.1514669154454205, -1....",180.315479,,-0.657631
4,100,4,"[-0.22887613765350334, -4.839075211421026, 3.7...",193.530331,,-8.549235
5,100,5,"[2.5208993944529796, 2.1134665696303783, 4.019...",199.838035,,0.487928
6,100,6,"[1.0201649333182106, -3.1863352560266005, 3.58...",182.244406,,2.241634
7,100,7,"[0.42532965680559265, 2.071468482942765, -4.93...",210.749391,,-20.657126
8,100,8,"[-0.8520207691894193, 1.840768084527907, 4.103...",171.300887,,-12.501119
9,100,9,"[3.371919856355608, -4.501398664978885, 1.4410...",222.578952,,-9.278973




Evolução concluída  - 100%

	 ---> RESULTADOS OBTIDOS


	 VISUALIZANDO OS RESULTADOS

Total execution time: 2.62 seconds
